### импортируем нужные библиотеки

In [1]:
import json
from tqdm import tqdm_notebook
import gzip
import pandas as pd
import sys

import json
from typing import Dict, List

Функция, которую используем для построения **OUT** 

In [7]:
def get_out_papers(papers: List[Dict],zip_num,context_dict=dict()) -> Dict:
    """
    Find all linked references
    :param paper:
    :param toks_in_context:
    :return:
    """

#     context_dict = dict()

    for paper in papers:
        if not paper:
            continue
        if not paper['paper_id']:
            continue
        if not (paper['grobid_parse'] or paper['latex_parse']):
            continue
        # only papers with text
        if paper['grobid_parse'] :
            if paper['latex_parse']:
                if not (paper['grobid_parse']['body_text'] or paper['latex_parse']['body_text']):
                    continue
            else:
                if not (paper['grobid_parse']['body_text'] ):
                    continue
        elif paper['latex_parse']:
            if paper['grobid_parse']:
                if not (paper['grobid_parse']['body_text'] or paper['latex_parse']['body_text']):
                    continue
            else:
                if not (paper['latex_parse']['body_text'] ):
                    continue
        
        # only papers with text and bib entries for find out references
        if paper['grobid_parse'] :
            if paper['latex_parse']:
                if not (paper['grobid_parse']['bib_entries'] or paper['latex_parse']['bib_entries']):
                    continue
            else:
                if not (paper['grobid_parse']['bib_entries'] ):
                    continue
        elif paper['latex_parse']:
            if paper['grobid_parse']:
                if not (paper['grobid_parse']['bib_entries'] or paper['latex_parse']['bib_entries']):
                    continue
            else:
                if not (paper['latex_parse']['bib_entries'] ):
                    continue
            
        if paper['grobid_parse'] and paper['grobid_parse']['bib_entries']:
            for key,values in paper['grobid_parse']['bib_entries'].items():
                value = values['links'] 

                if value :
                    if value == paper['paper_id']:
                        continue
                    #RPs adding
                    if paper['paper_id'] in context_dict:
                        if value not in context_dict[paper['paper_id']]['out']:
                            context_dict[paper['paper_id']]['out'].append(value)
                        if  context_dict[paper['paper_id']]['s2_pdf_hash'] == None:
                            context_dict[paper['paper_id']]['s2_pdf_hash'] = paper['s2_pdf_hash']
                        if  context_dict[paper['paper_id']]['zip_num'] == None:
                            context_dict[paper['paper_id']]['zip_num'] = zip_num
                    else:
                        # если нет данной статьи в словаре и у неё есть s2_pdf_hash
                        if paper['s2_pdf_hash']:
                            context_dict[paper['paper_id']] = {'out':[value],'s2_pdf_hash':paper['s2_pdf_hash'],'zip_num':zip_num}
                        # если нет данной статьи в словаре и у неё нет s2_pdf_hash, мб есть эта же статья далее с hash
                        else:
                            context_dict[paper['paper_id']] = {'out':[value],'s2_pdf_hash':None,'zip_num':None}
                else: # это нужно для дальнейшей оптимизации кода для быстрого построения in
                    #RPs adding
                    if paper['paper_id'] in context_dict:
                        continue
                    elif paper['grobid_parse']['body_text']:# те у данной статьи есть текст, но нет связанных bib_entries
                        context_dict[paper['paper_id']] = {'out':[],'s2_pdf_hash':None,'zip_num':None}
        
        if paper['latex_parse'] and paper['latex_parse']['bib_entries']:
            for key,values in paper['latex_parse']['bib_entries'].items():
                value = values['links'] 

                if value :
                    if value == paper['paper_id']:
                        continue
                    #RPs adding
                    if paper['paper_id'] in context_dict:
                        if value not in context_dict[paper['paper_id']]['out']:
                            context_dict[paper['paper_id']]['out'].append(value)
                        if  context_dict[paper['paper_id']]['s2_pdf_hash'] == None:
                            context_dict[paper['paper_id']]['s2_pdf_hash'] = paper['s2_pdf_hash']
                        if  context_dict[paper['paper_id']]['zip_num'] == None:
                            context_dict[paper['paper_id']]['zip_num'] = zip_num
                    else:
                        # если нет данной статьи в словаре и у неё есть s2_pdf_hash
                        if paper['s2_pdf_hash']:
                            context_dict[paper['paper_id']] = {'out':[value],'s2_pdf_hash':paper['s2_pdf_hash'],'zip_num':zip_num}
                        # если нет данной статьи в словаре и у неё нет s2_pdf_hash, мб есть эта же статья далее с hash
                        else:
                            context_dict[paper['paper_id']] = {'out':[value],'s2_pdf_hash':None,'zip_num':None}
                else: # это нужно для дальнейшей оптимизации кода для быстрого построения in
                    #RPs adding
                    if paper['paper_id'] in context_dict:
                        continue
                    elif paper['latex_parse']['body_text']:# те у данной статьи есть текст, но нет связанных bib_entries
                        context_dict[paper['paper_id']] = {'out':[],'s2_pdf_hash':None,'zip_num':None}
        

    return context_dict

#### Загрузим маленький датасет для проверки работы всех функций

In [ ]:
EXAMPLE_DATA_FILE = 'data/example_papers.jsonl'

In [8]:
all_contexts = []
all_papers = []
context_dict = dict()
with open(EXAMPLE_DATA_FILE, 'r') as f:
    for line in f:
        gorc_obj = json.loads(line)
        all_papers.append(gorc_obj)
        all_contexts += get_citation_contexts(gorc_obj)

In [9]:
from os import walk

f_zips = []
for (dirpath, dirnames, filenames) in walk('../../gorc/'):
    f_zips.extend(filenames)
    break

In [10]:
len(f_zips),f_zips

(10002,
 ['0.jsonl.gz',
  '1.jsonl.gz',
  '10.jsonl.gz',
  '100.jsonl.gz',
  '1000.jsonl.gz',
  '1001.jsonl.gz',
  '1002.jsonl.gz',
  '1003.jsonl.gz',
  '1004.jsonl.gz',
  '1005.jsonl.gz',
  '1006.jsonl.gz',
  '1007.jsonl.gz',
  '1008.jsonl.gz',
  '1009.jsonl.gz',
  '101.jsonl.gz',
  '1010.jsonl.gz',
  '1011.jsonl.gz',
  '1012.jsonl.gz',
  '1013.jsonl.gz',
  '1014.jsonl.gz',
  '1015.jsonl.gz',
  '1016.jsonl.gz',
  '1017.jsonl.gz',
  '1018.jsonl.gz',
  '1019.jsonl.gz',
  '102.jsonl.gz',
  '1020.jsonl.gz',
  '1021.jsonl.gz',
  '1022.jsonl.gz',
  '1023.jsonl.gz',
  '1024.jsonl.gz',
  '1025.jsonl.gz',
  '1026.jsonl.gz',
  '1027.jsonl.gz',
  '1028.jsonl.gz',
  '1029.jsonl.gz',
  '103.jsonl.gz',
  '1030.jsonl.gz',
  '1031.jsonl.gz',
  '1032.jsonl.gz',
  '1033.jsonl.gz',
  '1034.jsonl.gz',
  '1035.jsonl.gz',
  '1036.jsonl.gz',
  '1037.jsonl.gz',
  '1038.jsonl.gz',
  '1039.jsonl.gz',
  '104.jsonl.gz',
  '1040.jsonl.gz',
  '1041.jsonl.gz',
  '1042.jsonl.gz',
  '1043.jsonl.gz',
  '1044.jsonl.gz'

(10002,
 ['0.jsonl.gz',
  '1.jsonl.gz',
  '10.jsonl.gz',
  '100.jsonl.gz',
  '1000.jsonl.gz',
  '1001.jsonl.gz',
  '1002.jsonl.gz',
  '1003.jsonl.gz',
  '1004.jsonl.gz',
  '1005.jsonl.gz',
  '1006.jsonl.gz',
  '1007.jsonl.gz',
  '1008.jsonl.gz',
  '1009.jsonl.gz',
  '101.jsonl.gz',
  '1010.jsonl.gz',
  '1011.jsonl.gz',
  '1012.jsonl.gz',
  '1013.jsonl.gz',
  '1014.jsonl.gz',
  '1015.jsonl.gz',
  '1016.jsonl.gz',
  '1017.jsonl.gz',
  '1018.jsonl.gz',
  '1019.jsonl.gz',
  '102.jsonl.gz',
  '1020.jsonl.gz',
  '1021.jsonl.gz',
  '1022.jsonl.gz',
  '1023.jsonl.gz',
  '1024.jsonl.gz',
  '1025.jsonl.gz',
  '1026.jsonl.gz',
  '1027.jsonl.gz',
  '1028.jsonl.gz',
  '1029.jsonl.gz',
  '103.jsonl.gz',
  '1030.jsonl.gz',
  '1031.jsonl.gz',
  '1032.jsonl.gz',
  '1033.jsonl.gz',
  '1034.jsonl.gz',
  '1035.jsonl.gz',
  '1036.jsonl.gz',
  '1037.jsonl.gz',
  '1038.jsonl.gz',
  '1039.jsonl.gz',
  '104.jsonl.gz',
  '1040.jsonl.gz',
  '1041.jsonl.gz',
  '1042.jsonl.gz',
  '1043.jsonl.gz',
  '1044.jsonl.gz'

(10002,
 ['0.jsonl.gz',
  '1.jsonl.gz',
  '10.jsonl.gz',
  '100.jsonl.gz',
  '1000.jsonl.gz',
  '1001.jsonl.gz',
  '1002.jsonl.gz',
  '1003.jsonl.gz',
  '1004.jsonl.gz',
  '1005.jsonl.gz',
  '1006.jsonl.gz',
  '1007.jsonl.gz',
  '1008.jsonl.gz',
  '1009.jsonl.gz',
  '101.jsonl.gz',
  '1010.jsonl.gz',
  '1011.jsonl.gz',
  '1012.jsonl.gz',
  '1013.jsonl.gz',
  '1014.jsonl.gz',
  '1015.jsonl.gz',
  '1016.jsonl.gz',
  '1017.jsonl.gz',
  '1018.jsonl.gz',
  '1019.jsonl.gz',
  '102.jsonl.gz',
  '1020.jsonl.gz',
  '1021.jsonl.gz',
  '1022.jsonl.gz',
  '1023.jsonl.gz',
  '1024.jsonl.gz',
  '1025.jsonl.gz',
  '1026.jsonl.gz',
  '1027.jsonl.gz',
  '1028.jsonl.gz',
  '1029.jsonl.gz',
  '103.jsonl.gz',
  '1030.jsonl.gz',
  '1031.jsonl.gz',
  '1032.jsonl.gz',
  '1033.jsonl.gz',
  '1034.jsonl.gz',
  '1035.jsonl.gz',
  '1036.jsonl.gz',
  '1037.jsonl.gz',
  '1038.jsonl.gz',
  '1039.jsonl.gz',
  '104.jsonl.gz',
  '1040.jsonl.gz',
  '1041.jsonl.gz',
  '1042.jsonl.gz',
  '1043.jsonl.gz',
  '1044.jsonl.gz'

In [11]:
all_papers[0].keys()

dict_keys(['paper_id', 'metadata', 's2_pdf_hash', 'grobid_parse', 'latex_parse'])

dict_keys(['paper_id', 'metadata', 's2_pdf_hash', 'grobid_parse', 'latex_parse'])

dict_keys(['paper_id', 'metadata', 's2_pdf_hash', 'grobid_parse', 'latex_parse'])

In [12]:
all_papers[0]['metadata'].keys()

dict_keys(['title', 'authors', 'abstract', 'year', 'arxiv_id', 'acl_id', 'pmc_id', 'pubmed_id', 'doi', 'venue', 'journal'])

dict_keys(['title', 'authors', 'abstract', 'year', 'arxiv_id', 'acl_id', 'pmc_id', 'pubmed_id', 'doi', 'venue', 'journal'])

dict_keys(['title', 'authors', 'abstract', 'year', 'arxiv_id', 'acl_id', 'pmc_id', 'pubmed_id', 'doi', 'venue', 'journal'])

In [13]:
all_papers[0]['grobid_parse'].keys()

dict_keys(['abstract', 'body_text', 'ref_entries', 'bib_entries'])

dict_keys(['abstract', 'body_text', 'ref_entries', 'bib_entries'])

dict_keys(['abstract', 'body_text', 'ref_entries', 'bib_entries'])

In [14]:
all_papers[86]['latex_parse'].keys()

dict_keys(['abstract', 'body_text', 'ref_entries', 'bib_entries'])

dict_keys(['abstract', 'body_text', 'ref_entries', 'bib_entries'])

dict_keys(['abstract', 'body_text', 'ref_entries', 'bib_entries'])

In [15]:
ids_text_tex = [article['paper_id'] for article in all_papers if article['latex_parse'] and article['latex_parse']['body_text']]
len(ids_text_tex)

34

34

34

In [16]:
paper_ids = [article['paper_id'] for article in all_papers] # все paper_id статей

#### Выведем количество ссылок References найденных в grobid_parse & latex_parse 

In [22]:
for num,paper_id in enumerate(ids_text_tex):
    id_lst = paper_ids.index(paper_id)
    print(id_lst)#,all_papers[id_lst]['latex_parse']['bib_entries'])
    latex_links = [v['links'] for k,v in all_papers[id_lst]['latex_parse']['bib_entries'].items() if v['links']]
    if all_papers[id_lst]['grobid_parse']:
        grobid_links =  [v['links'] for k,v in all_papers[id_lst]['grobid_parse']['bib_entries'].items() if v['links']]
        if len(latex_links)> 0 and len(grobid_links) == 0:
            print('WOW!')
        print(f'latex_links = {len(latex_links)}| grobid_links = {len(grobid_links)}| together = {len(set(grobid_links+latex_links))}')
        print(10*'==')
        
    else:
        print(f'latex_links = {len(latex_links)}| grobid_links = 0| together = {len(set(latex_links))}')
        print(10*'==')
    

86
latex_links = 18| grobid_links = 21| together = 22
141
latex_links = 25| grobid_links = 28| together = 29
172
latex_links = 22| grobid_links = 23| together = 26
251
latex_links = 0| grobid_links = 0| together = 0
418
latex_links = 11| grobid_links = 11| together = 12
621
latex_links = 10| grobid_links = 9| together = 9
876
latex_links = 11| grobid_links = 11| together = 11
882
latex_links = 0| grobid_links = 0| together = 0
898
latex_links = 0| grobid_links = 0| together = 0
904
latex_links = 2| grobid_links = 2| together = 1
915
WOW!
latex_links = 17| grobid_links = 0| together = 17
920
latex_links = 0| grobid_links = 0| together = 0
932
latex_links = 0| grobid_links = 0| together = 0
934
latex_links = 10| grobid_links = 7| together = 12
945
latex_links = 16| grobid_links = 49| together = 49
947
latex_links = 0| grobid_links = 1| together = 1
951
latex_links = 0| grobid_links = 0| together = 0
955
latex_links = 8| grobid_links = 12| together = 13
958
latex_links = 0| grobid_links =

In [17]:
all_papers[-1]['grobid_parse']['bib_entries']

{'BIBREF0': {'ref_id': 'b0',
  'title': 'A mathematical theory of communication',
  'authors': [{'first': 'C',
    'middle': ['E'],
    'last': 'Shannon',
    'suffix': ''}],
  'year': 1948,
  'venue': 'Bell Labs Techn. J',
  'volume': '27',
  'issn': '',
  'pages': '379--423',
  'other_ids': {},
  'links': None},
 'BIBREF1': {'ref_id': 'b1',
  'title': 'The Shannon sampling theorem -its various extensions and applications: A tutorial review',
  'authors': [{'first': 'A', 'middle': ['J'], 'last': 'Jerri', 'suffix': ''}],
  'year': 1977,
  'venue': 'Proc. IEEE',
  'volume': '65',
  'issn': '',
  'pages': '1565--1596',
  'other_ids': {},
  'links': None},
 'BIBREF2': {'ref_id': 'b2',
  'title': 'Modern sampling theory: mathematics and applications',
  'authors': [{'first': 'J',
    'middle': ['J'],
    'last': 'Benedetto',
    'suffix': ''},
   {'first': 'P', 'middle': ['J S G'], 'last': 'Ferreira', 'suffix': ''}],
  'year': 2012,
  'venue': '',
  'volume': '',
  'issn': '',
  'pages': '

In [18]:
all_papers[-1]['latex_parse']['bib_entries']

{'BIBREF0': {'ref_id': 'BIBREF0',
  'title': 'A mathematical theory of communication',
  'authors': [{'first': 'C',
    'middle': ['E'],
    'last': 'Shannon',
    'suffix': ''}],
  'year': 1948,
  'venue': 'Bell Labs Techn. J',
  'volume': '27',
  'issn': '',
  'pages': '379--423',
  'other_ids': {},
  'links': '55379485'},
 'BIBREF1': {'ref_id': 'BIBREF1',
  'title': 'The Shannon sampling theorem - its various extensions and applications: A tutorial review',
  'authors': [{'first': 'A', 'middle': ['J'], 'last': 'Jerri', 'suffix': ''}],
  'year': 1977,
  'venue': 'Proc. IEEE',
  'volume': '65',
  'issn': '',
  'pages': '1565--1596',
  'other_ids': {},
  'links': '46235104'},
 'BIBREF2': {'ref_id': 'BIBREF2',
  'title': 'Modern sampling theory: mathematics and applications',
  'authors': [{'first': 'J',
    'middle': ['J'],
    'last': 'Benedetto',
    'suffix': ''},
   {'first': 'P', 'middle': ['J S G'], 'last': 'Ferreira', 'suffix': ''}],
  'year': 2012,
  'venue': '',
  'volume': ''

In [19]:
len([v['links'] for k,v in all_papers[-1]['latex_parse']['bib_entries'].items() if v['links']])

23

In [20]:
len([v['links'] for k,v in all_papers[-1]['grobid_parse']['bib_entries'].items() if v['links']])

0

**Как видим нужно использовать оба парсинга(grobid & latex) для нахождения reference**

**Иначе ухудшается покрытие! Мб ситуация когда в grobid вообще нет связанных ссылок, а в latex есть!**

In [23]:
del all_contexts
del all_papers

In [24]:
[file for file in f_zips if '.gz' not in file]

['s2orc-master.zip']

In [25]:
[file for file in f_zips if '.gz' in file][7360:7370]

['7622.jsonl.gz',
 '7623.jsonl.gz',
 '7624.jsonl.gz',
 '7625.jsonl.gz',
 '7626.jsonl.gz',
 '7627.jsonl.gz',
 '7628.jsonl.gz',
 '7629.jsonl.gz',
 '763.jsonl.gz',
 '763.jsonl.gz.f36663e6']

In [26]:
[file for file in f_zips if '.gz.' in file]

['763.jsonl.gz.f36663e6']

#### собираем датасет **out** по **БАТЧАМ**

In [29]:
# len_of_json_arxis = []

context_dict = dict()
# check_dict = dict()
epoch = -1
for file in tqdm_notebook([file for file in f_zips if '.gz' in file]):
    txt_input = "../../gorc/"+file
    
    zip_num = file.split('.')[0]
    epoch+=1
    all_articles = []
    
    with gzip.open(txt_input, "r") as read_file:
#         print(txt_input)
        try:
            json_list = list(read_file)
        except:
            print(f'ERROR epoch={epoch}')
#             epoch -=1
    for num, jsonchik in enumerate(json_list):
        try:
            result = json.loads(jsonchik)
#             context_dict = get_out_citation(result,context_dict)
            if result['grobid_parse'] != None or result['latex_parse'] != None:
                if result['grobid_parse']['body_text'] != None or result['latex_parse']['body_text'] != None:
                    all_articles.append(result)
        except:
            continue
    context_dict = get_out_papers(all_articles, zip_num, context_dict)
#     check_dict = get_out_papers_old(all_articles, zip_num, check_dict)
    read_file.close()
#     json_file = json.dumps(context_dict, separators=(',', ':'))
    if epoch % 1000 == 0:
        
        name_json = 'out_batches/context_dict_batch_'+str(epoch)+'.json' 
        with open(name_json, 'w+') as json_fil:
            json.dump(context_dict, json_fil, separators=(',', ':'), ensure_ascii=False)
            
        json_fil.close()
#     check_dict = get_check(all_articles, check_dict)
#     print(len(check_dict),len(context_dict),len(all_articles))
        print('epoch =',epoch,'|', len(context_dict),len(all_articles))
        context_dict = dict()

    # sum(len_of_json_arxis),len(all_articles)

epoch = 0 | 760 840
epoch = 1000 | 737314 857
epoch = 2000 | 738230 819
epoch = 3000 | 736506 843
epoch = 4000 | 738381 821
epoch = 5000 | 738145 854
epoch = 6000 | 738275 811
epoch = 7000 | 737087 877
ERROR epoch=7369
epoch = 8000 | 736190 777
epoch = 9000 | 739450 883
epoch = 10000 | 736237 841



In [30]:
epoch

10000

In [144]:
# if epoch % 1000 == 0:

#     name_json = 'out_batches/context_dict_batch_'+str(epoch)+'.json' 
#     with open(name_json, 'w+') as json_fil:
#         json.dump(context_dict, json_fil, separators=(',', ':'), ensure_ascii=False)

#     json_fil.close()
# #     check_dict = get_check(all_articles, check_dict)
# #     print(len(check_dict),len(context_dict),len(all_articles))
#     print('epoch =',epoch,'|', len(context_dict),len(all_articles))

epoch = 10000 | 655876 841


In [140]:
len(context_dict)

655876

In [43]:
import os
files = os.listdir('out_batches/')

In [44]:
for file in files:
    if 'context_dict_batch_' in file:
        if '_try' not in file:
            print(file)

context_dict_batch_0.json
context_dict_batch_1000.json
context_dict_batch_10000.json
context_dict_batch_2000.json
context_dict_batch_3000.json
context_dict_batch_4000.json
context_dict_batch_5000.json
context_dict_batch_6000.json
context_dict_batch_7000.json
context_dict_batch_8000.json
context_dict_batch_9000.json
context_dict_batch_0.json
context_dict_batch_1000.json
context_dict_batch_10000.json
context_dict_batch_2000.json
context_dict_batch_3000.json
context_dict_batch_4000.json
context_dict_batch_5000.json
context_dict_batch_6000.json
context_dict_batch_7000.json
context_dict_batch_8000.json
context_dict_batch_9000.json
context_dict_batch_0.json
context_dict_batch_1000.json
context_dict_batch_10000.json
context_dict_batch_2000.json
context_dict_batch_3000.json
context_dict_batch_4000.json
context_dict_batch_5000.json
context_dict_batch_6000.json
context_dict_batch_7000.json
context_dict_batch_8000.json
context_dict_batch_9000.json


#### сохраним paper_id всех статей из out (у всех есть текст статьи)

делается для дальнейшего упрощения датасета

In [60]:
all_ids_out_papers_with_text = []

for file in tqdm_notebook(files):
    if 'context_dict_batch_' in file:
        file = 'out_batches/'+file
        print(file)
        with open(file) as data_file:
            context_dict_batch = json.load(data_file)
        data_file.close()
        print(len(context_dict_batch))
        all_ids_out_papers_with_text+=list(context_dict_batch)


out_batches/context_dict_batch_0.json

out_batches/context_dict_batch_0.json


out_batches/context_dict_batch_0.json

760760760

out_batches/context_dict_batch_1000.json

out_batches/context_dict_batch_1000.json
out_batches/context_dict_batch_1000.json
737314737314737314


out_batches/context_dict_batch_10000.jsonout_batches/context_dict_batch_10000.jsonout_batches/context_dict_batch_10000.json


736237736237736237


out_batches/context_dict_batch_2000.json
out_batches/context_dict_batch_2000.jsonout_batches/context_dict_batch_2000.json

738230
738230738230

out_batches/context_dict_batch_3000.json
out_batches/context_dict_batch_3000.jsonout_batches/context_dict_batch_3000.json

736506736506736506


out_batches/context_dict_batch_4000.json
out_batches/context_dict_batch_4000.json
out_batches/context_dict_batch_4000.json
738381738381738381


out_batches/context_dict_batch_5000.json
out_batches/context_dict_batch_5000.jsonout_batches/context_dict_batch_5000.json

738145738145738145


out_batches/context_dict_batch_6000.json
out_batches/context_dict_batch_6000.jsono

In [62]:
len(all_ids_out_papers_with_text),len(set(all_ids_out_papers_with_text))

(7376575, 7376575)

(7376575, 7376575)

(7376575, 7376575)

In [63]:
with open('all_ids_out_papers_with_text.txt', 'w') as fout:
    for item in all_ids_out_papers_with_text:
        fout.write("%s\n" % item)
fout.close()

In [149]:
all_context_dict = dict()
for file in files:
    if 'context_dict_batch_' in file:
        file = 'out_batches/'+file
        print(file)
        with open(file) as data_file:
            context_dict_batch = json.load(data_file)
        data_file.close()
        all_context_dict.update(context_dict_batch)

out_batches/context_dict_batch_0.json
out_batches/context_dict_batch_1000.json
out_batches/context_dict_batch_10000.json
out_batches/context_dict_batch_2000.json
out_batches/context_dict_batch_3000.json
out_batches/context_dict_batch_4000.json
out_batches/context_dict_batch_5000.json
out_batches/context_dict_batch_6000.json
out_batches/context_dict_batch_7000.json
out_batches/context_dict_batch_8000.json
out_batches/context_dict_batch_9000.json


In [150]:
len(all_context_dict)

6576323

In [64]:
del context_dict

##### Идея уменьшения датасета:

если статья,на которую ссылается paper, есть в словаре, то мы её оставляем


In [66]:
all_out_ids = {item:1 for item in set(all_ids_out_papers_with_text)}

In [ ]:
all_ids_out_papers_with_text = []

for file in tqdm_notebook(files):
    if 'context_dict_batch_' in file:
        file = 'out_batches/'+file
        print(file)
        with open(file) as data_file:
            context_dict_batch = json.load(data_file)
        data_file.close()
        print(len(context_dict_batch))
        all_ids_out_papers_with_text+=list(context_dict_batch)


In [73]:
for file in tqdm_notebook(files):
    if 'context_dict_batch_' in file:
        out_file = 'out_batches/'+file
        print(out_file)
        with open(out_file) as data_file:
            context_dict_batch = json.load(data_file)
        data_file.close()
        
        dict_with_pdfs_likn = dict()
        for k,v in context_dict_batch.items():
            for out in v['out']:
                if out in all_out_ids:
                    if k in dict_with_pdfs_likn:
                        dict_with_pdfs_likn[k]['out'].append(out)
                    else:
                        dict_with_pdfs_likn[k] = {'out':[out],'s2_pdf_hash':context_dict_batch[k]['s2_pdf_hash']}
        
        del context_dict_batch
        
        name_json = 'out_dict_with_pdfs/'+ file
        print(name_json)
        with open(name_json, 'w+') as json_fil:
            json.dump(dict_with_pdfs_likn, json_fil, separators=(',', ':'), ensure_ascii=False)
            
        json_fil.close()
        del dict_with_pdfs_likn

out_batches/context_dict_batch_0.json


out_batches/context_dict_batch_0.json


out_batches/context_dict_batch_0.json
out_dict_with_pdfs/context_dict_batch_0.json
out_dict_with_pdfs/context_dict_batch_0.jsonout_dict_with_pdfs/context_dict_batch_0.json
out_batches/context_dict_batch_1000.json
out_batches/context_dict_batch_1000.json
out_batches/context_dict_batch_1000.json

out_dict_with_pdfs/context_dict_batch_1000.json
out_dict_with_pdfs/context_dict_batch_1000.jsonout_dict_with_pdfs/context_dict_batch_1000.json

out_batches/context_dict_batch_10000.json
out_batches/context_dict_batch_10000.json
out_batches/context_dict_batch_10000.json
out_dict_with_pdfs/context_dict_batch_10000.json
out_dict_with_pdfs/context_dict_batch_10000.jsonout_dict_with_pdfs/context_dict_batch_10000.json

out_batches/context_dict_batch_2000.json
out_batches/context_dict_batch_2000.json
out_batches/context_dict_batch_2000.json
out_dict_with_pdfs/context_dict_batch_2000.json
out_dict_with_pdfs/context_dict_batch_2000.jsonout_dict_with_pdfs/context_dict_batch_2000.json

out_batches/context_

In [ ]:
files = os.listdir('out_dict_with_pdfs/')

In [75]:
all_ids_out_papers_with_pdfs = []

for file in tqdm_notebook(files):
    if 'context_dict_batch_' in file:
        file = 'out_dict_with_pdfs/'+file
        print(file)
        with open(file) as data_file:
            context_dict_batch = json.load(data_file)
        data_file.close()
        print(len(context_dict_batch))
        all_ids_out_papers_with_pdfs+=list(context_dict_batch)


out_dict_with_pdfs/context_dict_batch_0.json
513
out_dict_with_pdfs/context_dict_batch_1000.json


out_dict_with_pdfs/context_dict_batch_0.json
513
out_dict_with_pdfs/context_dict_batch_1000.json


out_dict_with_pdfs/context_dict_batch_0.json
513
out_dict_with_pdfs/context_dict_batch_1000.json
507089
507089507089

out_dict_with_pdfs/context_dict_batch_10000.json
out_dict_with_pdfs/context_dict_batch_10000.json
out_dict_with_pdfs/context_dict_batch_10000.json
507151507151507151


out_dict_with_pdfs/context_dict_batch_2000.json
out_dict_with_pdfs/context_dict_batch_2000.jsonout_dict_with_pdfs/context_dict_batch_2000.json

508269508269508269


out_dict_with_pdfs/context_dict_batch_3000.json
out_dict_with_pdfs/context_dict_batch_3000.json
out_dict_with_pdfs/context_dict_batch_3000.json
507113507113507113


out_dict_with_pdfs/context_dict_batch_4000.json
out_dict_with_pdfs/context_dict_batch_4000.jsonout_dict_with_pdfs/context_dict_batch_4000.json

507798507798507798


out_dict_with_pdfs/context_dict_batch_5000.json
out_dict_with_pdfs/context_dict_batch_5000.json
out_dict_with_pdfs/context_dict_batch_5000.json
508413508413508413


out_dict_with_pdfs/context_dict_batch_6000.json
out_di

In [76]:
len(set(all_ids_out_papers_with_pdfs))

5076366

5076366

5076366

In [70]:
# with open('dict_with_pdfs_likn.json', 'w+') as json_fil:
#     json.dump(dict_with_pdfs_likn, json_fil, separators=(',', ':'), ensure_ascii=False)
# json_fil.close()    

In [12]:
# with open('dict_with_pdfs_likn.json') as data_file:
#     dict_with_pdfs_likn = json.load(data_file)
# data_file.close()

#### Можно также по батчам собирать dict_in, но это будет дольше

In [78]:
dict_in = dict()
for file in tqdm_notebook(files):
    if 'context_dict_batch_' in file:
        file = 'out_dict_with_pdfs/'+file
        print(file)
        with open(file) as data_file:
            context_dict_batch = json.load(data_file)
        data_file.close()

        for in_,val_in in tqdm_notebook(context_dict_batch.items()):
            for out_ in val_in['out']:
                if out_ in dict_in:
                    dict_in[out_]['in'].append(in_)
                    dict_in[out_]['in_hashes'].append(context_dict_batch[in_]['s2_pdf_hash'])
                else:
                    dict_in[out_] = {'in':[in_],'in_hashes':[context_dict_batch[in_]['s2_pdf_hash']]}
        
        del context_dict_batch

out_dict_with_pdfs/context_dict_batch_0.json


out_dict_with_pdfs/context_dict_batch_1000.json


out_dict_with_pdfs/context_dict_batch_0.json


out_dict_with_pdfs/context_dict_batch_1000.json


out_dict_with_pdfs/context_dict_batch_0.json


...

out_dict_with_pdfs/context_dict_batch_5000.jsonout_dict_with_pdfs/context_dict_batch_5000.json

out_dict_with_pdfs/context_dict_batch_5000.json


out_dict_with_pdfs/context_dict_batch_6000.json
out_dict_with_pdfs/context_dict_batch_6000.jsonout_dict_with_pdfs/context_dict_batch_6000.json



out_dict_with_pdfs/context_dict_batch_7000.json
out_dict_with_pdfs/context_dict_batch_7000.jsonout_dict_with_pdfs/context_dict_batch_7000.json



out_dict_with_pdfs/context_dict_batch_8000.json
out_dict_with_pdfs/context_dict_batch_8000.jsonout_dict_with_pdfs/context_dict_batch_8000.json



out_dict_with_pdfs/context_dict_batch_9000.json
out_dict_with_pdfs/context_dict_batch_9000.jsonout_dict_with_pdfs/context_dict_batch_9000.json



In [79]:
1+1

2

2

2

In [80]:
dict_in['73647128']

{'in': ['104172', '13713546', '131544897'],
 'in_hashes': ['73ed8076fc747e77c41845cb5f18b40ece350865',
  '61100c78f98937a82b6b6da12ffd147a35073506',
  'c37ebcdab464a77130fb1183b20c35232ad48d4b']}

{'in': ['104172', '13713546', '131544897'],
 'in_hashes': ['73ed8076fc747e77c41845cb5f18b40ece350865',
  '61100c78f98937a82b6b6da12ffd147a35073506',
  'c37ebcdab464a77130fb1183b20c35232ad48d4b']}

{'in': ['104172', '13713546', '131544897'],
 'in_hashes': ['73ed8076fc747e77c41845cb5f18b40ece350865',
  '61100c78f98937a82b6b6da12ffd147a35073506',
  'c37ebcdab464a77130fb1183b20c35232ad48d4b']}

#### по батчам

1. сделать для каждого out свой in
2. идти попарно по in и смотреть пересечения в paper_id (можно сразу найти id пересечения).(идем 1 и от 2 до 10, 2 от 3 до 10  и тд)
    - делать concat in к каждому id и удалять в dict_in c большим числом (1 & 2, удаляем в 2 и тд) 

In [ ]:
# dict_in = dict()
# for file in tqdm_notebook(files):
#     if 'context_dict_batch_' in file:
#         new_file = 'out_dict_with_pdfs/'+file
#         print(new_file)
#         with open(new_file) as data_file:
#             context_dict_batch = json.load(data_file)
#         data_file.close()

#         for in_,val_in in tqdm_notebook(context_dict_batch.items()):
#             for out_ in val_in['out']:
#                 if out_ in dict_in:
#                     dict_in[out_]['in'].append(in_)
#                     dict_in[out_]['in_hashes'].append(context_dict_batch[in_]['s2_pdf_hash'])
#                 else:
#                     dict_in[out_] = {'in':[in_],'in_hashes':[context_dict_batch[in_]['s2_pdf_hash']]}
        
#         del context_dict_batch

In [81]:
len(dict_in)

4133895

4133895

4133895

In [37]:
# len(dict_in) old

3813038

In [82]:
with open('dict_in_new.json', 'w+') as json_fil:
    json.dump(dict_in, json_fil, separators=(',', ':'), ensure_ascii=False)
json_fil.close()    

In [84]:
dict_in['73647128']['in']

['104172', '13713546', '131544897']

['104172', '13713546', '131544897']

['104172', '13713546', '131544897']

In [85]:
all_in_papers = set()
for k,v in dict_in.items():
    all_in_papers.add(k)
    for in_ in v['in']:
        all_in_papers.add(in_)

In [86]:
len(all_in_papers)

5986243

5986243

5986243

In [111]:
len(all_in_papers)

5626401

In [87]:
all_in_paper = list(all_in_papers)

In [88]:
with open('all_in_paper.txt', 'w') as f_in_file:
    for item in all_in_paper:
        f_in_file.write("%s\n" % item)
f_in_file.close()

In [ ]:
all_in_papers_ids = set(dict_in)

In [ ]:
with open('dict_in_keys_ids.txt', 'w') as f_in_file:
    for item in all_in_papers_ids:
        f_in_file.write("%s\n" % item)
f_in_file.close()